# CS 541-A-Homework 1
## K nearest neighbors and distance metrics


---


### ***Fill your details below***
### Name: Adib Osmany
### CWID: 20013762
### Email ID: aosmany@stevens.edu
### References: ***Cite your references here***


---
### Submission guidelines:

#### 1. Submit this notebook along with its PDF version. You can do this by clicking File->Print->"Save as PDF"

#### 2. Name the file as "<mailID_HWnumber.extension>". For example, mailID is abcdefg @stevens.edu then name the files as abcdefg_HW1.ipynb and abcdefg_HW1.pdf.

#### 3. Please do not Zip your files.


---



### Install prerequisites

In [2]:
!pip install sentencepiece datasets
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-2kcsy3ob
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-2kcsy3ob
  Resolved https://github.com/huggingface/transformers to commit c8d98405a8f7b0e5d07391b671dcc61bb9d7bad5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Import relevant libraries

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

### Load dataset

In [4]:
# Load Financial banking77 dataset
train_dataset, test_dataset = load_dataset('banking77', split=['train', 'test'])
print("Train\n", train_dataset)
print("Test\n", test_dataset)

# Accessing example from the training set
example = train_dataset[0]
print("Example from the training set:")
print(example)

# Zhaozhuo: Just organize a bit.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Train
 Dataset({
    features: ['text', 'label'],
    num_rows: 10003
})
Test
 Dataset({
    features: ['text', 'label'],
    num_rows: 3080
})
Example from the training set:
{'text': 'I am still waiting on my card?', 'label': 11}


### Load tokenizer

In [5]:
# Define tokenzier
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2", use_fast=True)

### Tokenize dataset

In [6]:
# Tokenize the training and testing set
tokenized_trainset=tokenizer(train_dataset["text"])['input_ids']
train_labels=train_dataset["label"]

tokenized_testset=tokenizer(test_dataset["text"])['input_ids']
test_labels=test_dataset["label"]

## Q1. (40 Points) Write a function to compute Jaccard distance between lists

In [7]:
def jaccard_distance(list1, list2):
  '''
  Compute the Jaccard distance between two lists.
  '''
  l1=set(list1)
  l2=set(list2)
  similarity= l1.intersection(l2)
  denominator=l1.union(l2)

  return (1-(len(similarity)/len(denominator)))


  return #return the Jaccard distance

### Test script to check the Jaccard distance calculation function

In [8]:
test_distance=jaccard_distance([1,2],[1,3,4])

if(test_distance==0.75):
  print("Q1: Correct")
else:
  print("Q1: Wrong")

Q1: Correct


## Q2(a) (15 Points) Write code to implement the K nearest neighbors (KNN) algorithm with ***Jaccard*** distance. Use K=1 to compute accuracy on test set.

### Note: Please code this section from scratch, do not use KNN implementations from libraries such as scikit-learn.

### Hint 1: K=1 means that only one nearest neighbor votes for the sample's label. Find the sample in the trainset which is closest to the test sample and predict it's label as the test sample's label.

### Hint 2: Use the tokenized train set and tokenized test set

In [9]:
total_num_correct = 0
number_of_testing_cases = len(tokenized_testset)

'''
Add KNN code here

returns nearest neigbor's label
'''
def KNN(sample):
  results=[]
  for i in range(len(tokenized_trainset)):
    point=tokenized_trainset[i]
    label=train_labels[i]
    results.append((jaccard_distance(sample, point), label))

  results.sort()
  return results[0][1]


'''testing for accuracy
   Note: Runtime is around 2m 45s
   Accuracy: 75%                 '''
for i in range(len(tokenized_testset)):
  sample=tokenized_testset[i]
  correct_label=test_labels[i]
  output=KNN(sample)

  if output==correct_label:
    total_num_correct=total_num_correct+1

print("Accuracy:",total_num_correct/number_of_testing_cases)


Accuracy: 0.7532467532467533


## Q2(b) (15 Points) Please print out three samples from the test set along with their nearest neighbor from the train set. Discuss your observations.

In [23]:
'''Three samples'''
sample1=tokenized_testset[100]
sample2=tokenized_testset[200]
sample3=tokenized_testset[800]

'''This KNN function returns the nearest neighbor, while the previous KNN function returns the nearest neighbor's label'''
def KNN_neighbor(sample):
  results=[]
  for i in range(len(tokenized_trainset)):
    point=tokenized_trainset[i]
    results.append((jaccard_distance(sample, point), point))

  results.sort()
  return results[0][1]

'''sample 1'''
print("The first test sample is ")
print(sample1)
print("Its nearest neighbor is ")
print(KNN_neighbor(sample1))
print("Its predicted label is: ", end="")
print(KNN(sample1))
print("Its actual label is: ", end="")
print(test_labels[100])
print()

'''sample 2'''
print("The second test sample is ")
print(sample2)
print("Its nearest neighbor is ")
print(KNN_neighbor(sample2))
print("Its predicted label is: ", end="")
print(KNN(sample2))
print("Its actual label is: ", end="")
print(test_labels[200])
print()

'''sample 3'''
print("The third test sample is ")
print(sample3)
print("Its nearest neighbor is ")
print(KNN_neighbor(sample3))
print("Its predicted label is: ", end="")
print(KNN(sample3))
print("Its actual label is: ", end="")
print(test_labels[800])
print()

print("Observation:")
print("""All three sample labels appear to have been predicted correctly.
This is not suprising since the our KNN algorithm has a 75% acuuracy rate, so
it would have been harder to find a sample label that was not predicted correctly.
One thing I noticed is that the samples dont always have the same size, as
shown in second test, where the test sample has a size of 11, while its nearest
neighbor has a size of 12. Another thing I noticed is that the samples and their
nearest neighbor sometimes do not have elements at the same index. For example,
the second sample shared the element '10632', but the element in the sample was at
index 2, while the element at the nearest neighbor was at index 4. The order of
which the elements reside must not matter since we used Jaccards distance for
our KNN algorithm.
""")


The first test sample is 
[101, 2054, 2003, 1996, 2918, 1997, 1996, 3863, 6165, 1029, 102]
Its nearest neighbor is 
[101, 2054, 2003, 1996, 3120, 1997, 2115, 3863, 6165, 1029, 102]
Its predicted label is: 32
Its actual label is: 32

The second test sample is 
[101, 2026, 10632, 2140, 2003, 2145, 14223, 1012, 2339, 1029, 102]
Its nearest neighbor is 
[101, 2339, 2003, 2026, 10632, 2140, 2000, 5356, 2145, 14223, 1029, 102]
Its predicted label is: 46
Its actual label is: 46

The third test sample is 
[101, 1045, 3477, 2098, 2007, 1037, 4003, 1998, 2001, 5338, 2019, 4469, 7408, 102]
Its nearest neighbor is 
[101, 1045, 3477, 2098, 2007, 1996, 4003, 1998, 2045, 2001, 2019, 4469, 7408, 1029, 102]
Its predicted label is: 15
Its actual label is: 15

Observation:
All three sample labels appear to have been predicted correctly.
This is not suprising since the our KNN algorithm has a 75% acuuracy rate, so
it would have been harder to find a sample label that was not predicted correctly.
One thing

## Q3(a) (10 Points) Below, we have provided a method to generate embeddings. Please study the code and generate embeddings for the training set. Also complete the function to calculate eucleadian distance

In [11]:
# Function to calculate Euclidean distance
import math
def Euclidean_distance(array1, array2):
  '''
  Compute the Euclidean distance between two arrays.
  '''
  total=0
  for i in range(len(array1)):
    total=total+pow(array1[i]-array2[i],2)

  return math.sqrt(total)

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = test_dataset["text"]
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
testset_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
testset_embeddings = F.normalize(testset_embeddings, p=2, dim=1)

print("testset_embeddings:")
print(testset_embeddings)

'''
Write your code here

Used a smaller training set to avoid computation issues
new training set size: 4000
'''
sentences = train_dataset["text"]
sentences= sentences[:1000]+sentences[3000:4000]+sentences[6000:7000]+sentences[9000:10000]
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)
trainset_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
trainset_embeddings = F.normalize(trainset_embeddings, p=2, dim=1)
print("trainset_embeddings:")
print(trainset_embeddings)


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


testset_embeddings:
tensor([[-0.0102,  0.0188, -0.0575,  ...,  0.0964, -0.0615, -0.0092],
        [-0.0428, -0.0263,  0.0400,  ..., -0.0296, -0.0079,  0.0679],
        [-0.0337,  0.0374,  0.0194,  ...,  0.0616, -0.0190,  0.0224],
        ...,
        [ 0.1096, -0.0527, -0.0050,  ..., -0.1261,  0.0224, -0.0335],
        [ 0.0984, -0.0408,  0.0406,  ..., -0.0376,  0.0214, -0.0130],
        [-0.0144,  0.0701, -0.0568,  ..., -0.0215,  0.0555,  0.0137]])
trainset_embeddings:
tensor([[-0.0354, -0.0421, -0.0028,  ..., -0.1048, -0.0466,  0.0028],
        [ 0.0226, -0.0135,  0.0243,  ...,  0.0013, -0.0254,  0.0173],
        [-0.0460, -0.0199, -0.0015,  ..., -0.0797,  0.0138,  0.0683],
        ...,
        [ 0.0324, -0.0624,  0.0013,  ...,  0.0347, -0.0022,  0.0190],
        [-0.0045,  0.0190,  0.0331,  ..., -0.0260,  0.0395,  0.0020],
        [ 0.0193, -0.0252,  0.0279,  ..., -0.0542,  0.0313,  0.0475]])


## Q3(b) (10 Points) Please write a KNN classifier that uses the ***Euclidean*** distance between ***Embeddings*** of samples. Predict the labels for the test set and printout the accuracy.

In [27]:
import random
total_num_correct = 0
number_of_testing_cases = 50

'''Decreased the embedded training set even more to avoid computational issues'''
train_accuracy=trainset_embeddings[:1000]

def KNN_ecldn(sample):
  results=[]
  for i in range(len(train_accuracy)):
    point=train_accuracy[i]
    label=train_labels[i]
    results.append((Euclidean_distance(sample, point), label))

  results.sort()
  return results[0][1]

'''testing for accuracy
   # of tests: 50
   Note: Runtime is around 4m 54s
   Accuracy: 22%                 '''

'''selected 50 test embedded sets to avoid runtime issues when compared to running all the sets
  Selected by random for fairness'''
for i in range(50):
  j=random.randint(0,999)
  sample=testset_embeddings[j]
  correct_label=test_labels[j]
  output=KNN_ecldn(sample)

  if output==correct_label:
    total_num_correct=total_num_correct+1

print("Accuracy:",total_num_correct/number_of_testing_cases)

Accuracy: 0.22


## Q3(c) (10 Points) Printout the same three samples that you used for Q2(b) along with their nearest neighbors found in Q3(b). Discuss your observations.




In [35]:
'''Three samples'''
sample1=tokenized_testset[100]
sample2=tokenized_testset[200]
sample3=tokenized_testset[800]

'''This KNN function returns the nearest neighbor, while the previous KNN function returns the nearest neighbor's label'''
def KNN_neighbor_q3(sample):
  results=[]
  for i in range(len(train_accuracy)):
    point=train_accuracy[i]
    unembedded=tokenized_trainset[i]
    results.append((Euclidean_distance(sample, point), unembedded))
  results.sort()
  return results[0][1]



'''The samples are printed out in their tokenized version instead of their embedded version to better analysis the observations'''

'''sample 1'''
print("The first test sample is ")
print(sample1)
print("Its nearest neighbor is ")
print(KNN_neighbor_q3(testset_embeddings[100]))
print("Its predicted label is: ", end="")
print(KNN_ecldn(testset_embeddings[100]))
print("Its actual label is: ", end="")
print(test_labels[100])
print()

'''sample 2'''
print("The second test sample is ")
print(sample2)
print("Its nearest neighbor is ")
print(KNN_neighbor_q3(testset_embeddings[200]))
print("Its predicted label is: ", end="")
print(KNN_ecldn(testset_embeddings[200]))
print("Its actual label is: ", end="")
print(test_labels[200])
print()

'''sample 3'''
print("The third test sample is ")
print(sample3)
print("Its nearest neighbor is ")
print(KNN_neighbor_q3(testset_embeddings[800]))
print("Its predicted label is: ", end="")
print(KNN_ecldn(testset_embeddings[800]))
print("Its actual label is: ", end="")
print(test_labels[800])
print()

print("Observation:")
print(""" The samples and nearest neigbors are printed out in their tokenized version instead of their
embedded version to better analyze the observations. 2 out of 3 samples appear to have been predicted correctly for thier labels.
This is surprising since our accuracy was shown to be low. However, the accuracy was
only low because not all of the data was used for this prediction. The data sets had to
be cut significantly to avoid computational issues, so in reality, if all the data was
used properly, we would have a more appropriate accuracy reading.
One thing I noticed is that the nearest neighbor for the samples are not the same nearest
neighbor as the ones in Q2b. This must be because of our use of Euclidean distance instead
of jaccard distance. The sample that was not predicted correctly was sample 3. This may have been
due to the fact that this sample was closer to where we shortened our test, so possible data points
were not used.
""")


The first test sample is 
[101, 2054, 2003, 1996, 2918, 1997, 1996, 3863, 6165, 1029, 102]
Its nearest neighbor is 
[101, 2006, 2054, 2079, 2017, 2918, 2115, 3863, 6165, 1029, 102]
Its predicted label is: 32
Its actual label is: 32

The second test sample is 
[101, 2026, 10632, 2140, 2003, 2145, 14223, 1012, 2339, 1029, 102]
Its nearest neighbor is 
[101, 2026, 10534, 2003, 14223, 1010, 2339, 1029, 102]
Its predicted label is: 46
Its actual label is: 46

The third test sample is 
[101, 1045, 3477, 2098, 2007, 1037, 4003, 1998, 2001, 5338, 2019, 4469, 7408, 102]
Its nearest neighbor is 
[101, 2339, 2038, 2026, 4003, 2042, 5338, 2019, 4469, 9044, 1029, 102]
Its predicted label is: 34
Its actual label is: 15

Observation:
 The samples and nearest neigbors are printed out in their tokenized version instead of their 
embedded version to better analyze the observations. 2 out of 3 samples appear to have been predicted correctly for thier labels.
This is surprising since our accuracy was show